<a href="https://colab.research.google.com/github/xerxes786/Deep-Learning/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 5.5 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [9]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [10]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.8725000023841858

Best val_accuracy So Far: 0.8889999985694885
Total elapsed time: 00h 04m 39s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        13872     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 96)                2654304   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2,669,466
Trainable params: 2,669,466
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 15s 8ms/step - loss: 0.2497 - accuracy: 0.9083 - val_loss: 0.3784 - val_accuracy: 0.8682
Epoch 5/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2249 - accuracy: 0.9182 - val_loss: 0.3998 - val_accuracy: 0.8823
Epoch 6/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.2063 - accuracy: 0.9254 - val_loss: 0.4658 - val_accuracy: 0.8812
Epoch 7/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1897 - accuracy: 0.9322 - val_loss: 0.4426 - val_accuracy: 0.8843
Epoch 8/10
1688/1688 [==============================] - 14s 9ms/step - loss: 0.1715 - accuracy: 0.9401 - val_loss: 0.5682 - val_accuracy: 0.8780
Epoch 9/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1562 - accuracy: 0.9451 - val_loss: 0.5684 - val_accuracy: 0.8798
Epoch 10/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1498 - accuracy: 0.9484 - val_loss: 0.5982 - val_a